In [ ]:
pip install PyPDF2

In [ ]:
pip install requests

In [ ]:
# Notas antigas da AVENUE Até maio de 2021 Apenas analise dos dados # Conferencia Pesquisa por ticket ou algo especifico dentro dos PDFs
import os
import PyPDF2
import re

pasta = 'C:/Documents/notas/'

#pattern = r"(.*)\n(.*)\n(.*)\n(Sell)\s+([-\d,.]+)\s+([\d,.]+)\s+(\d{1,2}\/\d{1,2}\/\d{4})\s+(\d{1,2}\/\d{1,2}\/\d{4})" # confere Sell
#pattern = r"(.*)\n(.*)\n(.*)\n(Sell)\s+([-\d,.]+)\s+([\d,.]+)\s+(\d{1,2}\/\d{1,2}\/\d{4})\s+(\d{1,2}\/\d{1,2}\/\d{4})" # ticker COP |Buy
pattern = r"(.*)\n(.*)\n(.*)\n(Buy|Sell)\s+([-\d,.]+)\s+([\d,.]+)\s+(\d{1,2}\/\d{1,2}\/\d{4})\s+(\d{1,2}\/\d{1,2}\/\d{4})"
# pdf
contador_pdf = 0
dados_extraidos = {}
for root, dirs, files in os.walk(pasta):
    for filename in files:
        if filename.endswith('.pdf'):
            contador_pdf += 1
            caminho_pdf = os.path.join(root, filename)
            with open(caminho_pdf, 'rb') as pdf_file:
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                dados_por_pdf = []
                for page_num in range(len(pdf_reader.pages)):
                    pagina = pdf_reader.pages[page_num]
                    texto = pagina.extract_text()
                    matches = re.findall(pattern, texto)
                    for match in matches:
                        # print(texto)
                        # print(match)
                        # NOTA ANTIGA:
                        sym = match[0]
                        buy_sell = match[3].upper()
                        qty = match[4].replace('-', '')
                        price = match[5].replace(',', '')
                        trade_date = match[6]
                        month, day, year = trade_date.split('/')
                        trade_date_br = f"{day.zfill(2)}/{month.zfill(2)}/{year}"
                        qty_br = f"{float(qty):.8f}".replace(',', '').replace('.', ',')
                        price_br = f"{float(price):.2f}".replace(',', '').replace('.', ',')
                        dados_append = {
                            'Trade_Date': trade_date_br,
                            'QTY': qty_br,
                            'SYM': sym,
                            'PRICE': price_br,
                            'buy_sell': buy_sell
                        }
                        dados_por_pdf.append(dados_append)
                        print(dados_append)
                dados_extraidos[filename] = dados_por_pdf

# for pdf, dados in dados_extraidos.items():
#     print(f"Dados extraídos de {pdf}:")
#     for dado in dados:
#         print(dado)
print(pasta)
print(len(dados_extraidos.items()))
print(f"Total de arquivos PDF: {contador_pdf}")

In [ ]:
# GET Nessa parte só organiza os Simbolos dos tickers com o código correto e carrega em um arquivo.
# Notas antigas avenue
import os
import PyPDF2
import re
import requests
import json

pasta = 'C:/Documents/notas/2020/'
pattern = r"(.*)\n(.*)\n(.*)\n(Buy|Sell)\s+([-\d,.]+)\s+([\d,.]+)\s+(\d{1,2}\/\d{1,2}\/\d{4})\s+(\d{1,2}\/\d{1,2}\/\d{4})" # (.*)\n linha inteira até a quebra

contador_pdf = 0
dados_extraidos = {}
for root, dirs, files in os.walk(pasta):
    for filename in files:
        if filename.endswith('.pdf'):
            contador_pdf += 1
            caminho_pdf = os.path.join(root, filename)
            with open(caminho_pdf, 'rb') as pdf_file:
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                dados_por_pdf = []
                for page_num in range(len(pdf_reader.pages)):
                    pagina = pdf_reader.pages[page_num]
                    texto = pagina.extract_text()
                    matches = re.findall(pattern, texto)
                    for match in matches:
                        # NOTA ANTIGA:
                        sym = match[0]
                        buy_sell = match[3]
                        qty = match[4].replace('-', '')
                        price = match[5].replace(',', '')
                        trade_date = match[6]
                        month, day, year = trade_date.split('/')
                        trade_date_br = f"{day.zfill(2)}/{month.zfill(2)}/{year}"
                        qty_br = f"{float(qty):.8f}".replace(',', '').replace('.', ',')
                        price_br = f"{float(price):.2f}".replace(',', '').replace('.', ',')
                        dados_append = {
                            'Trade_Date': trade_date_br,
                            'QTY': qty_br,
                            'SYM': sym,
                            'PRICE': price_br,
                            'buy_sell': buy_sell
                        }
                        dados_por_pdf.append(dados_append)
                        # print(dados_append)
                dados_extraidos[filename] = dados_por_pdf

sym_unicos = set()

# Percorre os dados extraídos para coletar os SYMs
for pdf, dados in dados_extraidos.items():
    for dado in dados:
        sym_unicos.add(dado['SYM'])

print("SYMs encontrados sem repetição: " + str(len(sym_unicos)))

## abre arquivo com sym Simbols JSON ticker, id, tipo (Stock/Reit)
filename = "resultados_simbolos_stock_investidor10.txt"
resultados_arq_json = []
try:
    with open(filename, 'r') as file:
        resultados_arq_json = json.load(file)
except Exception:
    resultados_arq_json = []

tickers_no_json = {item['ticker'] for item in resultados_arq_json}

# for item in resultados_arq_json:
#     ticker = item.get('ticker')
#     id_ = item.get('id')
#     tipo = item.get('tipo')
#     print(f'Ticker: {ticker}, ID: {id_}, Tipo: {tipo}')

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0",
    "Accept": "application/json",
}

# EtfInternational Treasure FixedIncome Fund Fii Crypto Stock Reit StockBdr Etf
def consulta(tipo, sym):
    url = f"https://investidor10.com.br/api/buscar/{tipo}?term={sym}&_type=query&q={sym}"
    response2 = requests.get(url, headers=headers)
    if response2.status_code == 200:
        data2 = response2.json()
        if data2 and isinstance(data2, list):
            print(f"""
    {{
        "ticker": "{data2[0].get('name').split(' ')[0]}",
        "id": {data2[0].get('id')},
        "tipo": "{tipo}"
    }}
                    """)
            #sym_ids.append({'ticker': sym, 'id': item.get('id'), 'tipo': 'Stock'})
    else:
        return False
    return True

sym_ids = []
for sym in sym_unicos:
    if sym not in tickers_no_json: #.values():
        url = f"https://investidor10.com.br/api/buscar/Stock?term={sym}&_type=query&q={sym}"
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data and isinstance(data, list):
                if len(data) > 1:
                    encontrou = False
                    for item in data:
                        name_res = item.get('name').split(" ")
                        if name_res[0] == sym:
                           sym_ids.append({'ticker': sym, 'id': item.get('id'), 'tipo': 'Stock'})
                           encontrou = True
                           break
                    
                    if not encontrou:
                        print("Foi encontrado mais de um Ticker com este simbolo, confira e edite no documento de simbolos o ID correto: " + sym)
                        consulta('Reit', sym)
                        # print(f"ID: {item.get('id')}, Nome: {item.get('name')}")
                else:
                    sym_ids.append({'ticker': sym, 'id': data[0].get('id'), 'tipo': 'Stock'})
                # sym_ids[sym] = data[0].get('id')
            else:
                print("Esse simbolo não foi encontrado como Stock, confira e edite manualmente o arquivo de símbolo: " + sym)
                consulta('Reit', sym)
                consulta('EtfInternational', sym)
        else:
            print(f"Erro: {response.status_code} - {response.text}")

if sym_ids:
    print("Gravando arquivo de simbolos/tickers")
    resultados_arq_json.extend(sym_ids)
    with open(filename, 'w') as file:
        json.dump(resultados_arq_json, file, indent=4)

print(pasta)

# for pdf, dados in dados_extraidos.items():
#     print(f"Dados extraídos de {pdf}:")
#     for dado in dados:
#         print(dado)
#     print("="*50)


In [ ]:
# Importar notas
# PDF Extrai dados e insere no Investidor10
# Notas antigas avenue
import PyPDF2
import os
import re
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

# TODO Informar:
usuario = "usuario@mail.com"
senha = "xxxxxxxxxxx"

pasta = 'C:/Documents/notas/2020/'

pattern = r"(.*)\n(.*)\n(.*)\n(Buy|Sell)\s+([-\d,.]+)\s+([\d,.]+)\s+(\d{1,2}\/\d{1,2}\/\d{4})\s+(\d{1,2}\/\d{1,2}\/\d{4})"

driver = webdriver.Chrome()

def login_investidor10():
    driver.get("https://investidor10.com.br/login")
    driver.find_element(By.NAME, "email").send_keys(usuario)
    driver.find_element(By.NAME, "password").send_keys(senha)
    driver.find_element(By.NAME, "password").send_keys(Keys.ENTER)
    time.sleep(1)
    driver.get("https://investidor10.com.br/carteiras/lancamentos/1274614/")
    time.sleep(3)
    driver.execute_script("MyWallets.openPopUp('add_ticker_entry', true, {'ticker_type' : 'Ticker'})") # PopUp INCLUIR LANÇAMENTO

def envia_dados_investidor10(dados):
    print(dados)
    time.sleep(1)
    select_element = driver.find_element(By.ID, 'ticker_type')
    select = Select(select_element)
    select.select_by_value(dados['ticker_type'])

    valor_buy_sell = 'BUY' if dados['type'] == 'Buy' else 'SELL' # .upper()
    driver.execute_script(f"document.getElementsByName('type')[0].value = '{valor_buy_sell}';")

    driver.execute_script(f"""
    var select = document.getElementsByName('ticker')[0];
    var option = document.createElement('option');
    option.value = '{dados['ticker']}';
    option.text = '{dados['SYM']}';
    select.add(option);
    select.value = '{dados['ticker']}';
    """)
    driver.find_element(By.NAME, "date").clear()
    driver.find_element(By.NAME, "date").send_keys(dados['date'])
    driver.find_element(By.NAME, "date").send_keys(Keys.TAB)
    time.sleep(1)
    driver.find_element(By.NAME, "qty").clear()
    driver.find_element(By.NAME, "qty").send_keys(dados['qty'])
    driver.find_element(By.NAME, "qty").send_keys(Keys.TAB)
    time.sleep(0.5)
    driver.find_element(By.NAME, "price").clear()
    driver.find_element(By.NAME, "price").send_keys(dados['price'])
    driver.find_element(By.NAME, "price").send_keys(Keys.ENTER)
    time.sleep(1)
    driver.find_element(By.XPATH, "//button[text()='Adicionar outro ativo']").click()

contador_pdf = 0
dados_extraidos = {}
for root, dirs, files in os.walk(pasta):
    for filename in files:
        if filename.endswith('.pdf'):
            contador_pdf += 1
            caminho_pdf = os.path.join(root, filename)
            with open(caminho_pdf, 'rb') as pdf_file:
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                dados_por_pdf = []
                for page_num in range(len(pdf_reader.pages)):
                    pagina = pdf_reader.pages[page_num]
                    texto = pagina.extract_text()
                    matches = re.findall(pattern, texto)
                    for match in matches:
                        # NOTA ANTIGA:
                        sym = match[0]
                        buy_sell = match[3]
                        qty = match[4].replace('-', '')
                        price = match[5].replace(',', '')
                        trade_date = match[6]
                        month, day, year = trade_date.split('/')
                        trade_date_br = f"{day.zfill(2)}/{month.zfill(2)}/{year}"
                        qty_br = f"{float(qty):.8f}".replace(',', '').replace('.', ',')
                        price_br = f"{float(price):.2f}".replace(',', '').replace('.', ',')
                        dados_append = {
                            'Trade_Date': trade_date_br,
                            'QTY': qty_br,
                            'SYM': sym,
                            'PRICE': price_br,
                            'buy_sell': buy_sell
                        }
                        dados_por_pdf.append(dados_append)
                dados_extraidos[filename] = dados_por_pdf


def buscar_por_ticker(ticker_procurado, lista):
    for item in lista:
        if item["ticker"] == ticker_procurado:
            return item
    return None

filename = "resultados_simbolos_stock_investidor10.txt"
resultados_arq_json = []
# try:
with open(filename, 'r') as file:
    resultados_arq_json = json.load(file)
# except Exception:
#     resultados_arq_json = []

login_investidor10()

# Percorre dados_extraidos e Envia para o investidor10
for pdf, dados in dados_extraidos.items():
    # print(f"Dados extraídos de {pdf}:")
    for dado in dados:
        #id_found = next((id for id, sym in resultados_arq.items() if sym == dado['SYM']), None)
        ticker_base = buscar_por_ticker(dado['SYM'], resultados_arq_json)
        if ticker_base is not None:
            data_resultados = {
                "ticker_type": ticker_base['tipo'],
                "user_wallet_id": "1274614",
                "type": dado['buy_sell'],
                "source": "MANUAL",
                "_token": "QIrfkhZ8Z1qwKlqEYJWcDXvMRA0PwHYyR7k1rk1J",
                "date": dado['Trade_Date'],
                "qty": dado['QTY'],
                "ticker": ticker_base['id'],
                "price": dado['PRICE'],
                "SYM": dado['SYM']
            }
            envia_dados_investidor10(data_resultados)
        else:
            print(f"{dado['SYM']} não encontrado: ")
            print(dado)
            # print("="*50)
    # break

print(len(dados_extraidos.items()))